In [2]:
import pandas as pd
import time
from nltk import tokenize
from allennlp_models.pretrained import load_predictor

In [21]:
predictor = load_predictor("coref-spanbert")
predictor._model = predictor._model.cuda()

error loading _jsonnet (this is expected on Windows), treating C:\Users\Xianglong\Desktop\UMich\Courses\EECS 498 Conversation AI\Sirious\Question Answering Model\env\Lib\site-packages\allennlp_models\modelcards\coref-spanbert.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\Xianglong\Desktop\UMich\Courses\EECS 498 Conversation AI\Sirious\Question Answering Model\env\Lib\site-packages\allennlp_models\modelcards\evaluate_rc-lerc.json as plain json
lerc is not a registered model.
error loading _jsonnet (this is expected on Windows), treating C:\Users\Xianglong\Desktop\UMich\Courses\EECS 498 Conversation AI\Sirious\Question Answering Model\env\Lib\site-packages\allennlp_models\modelcards\generation-bart.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\Xianglong\Desktop\UMich\Courses\EECS 498 Conversation AI\Sirious\Question Answering Model\env\Lib\site-packages\allennlp_models\modelcards\glove-sst.json as p

In [4]:
lectures = pd.read_csv('context/raw_text.csv')

In [31]:
lectures['text'] = lectures['text'].apply(lambda text: text.replace('\n', ' '))

In [32]:
lectures.head()

,text,label
0,The following content is provided under a Crea...,Calculus
1,"In this sequence of segments, we review some m...",Probability
2,The following content is provided under a Crea...,CS
3,The following content is provided under a Crea...,Algorithms
4,The following content is provided under a Crea...,Algorithms


In [33]:
lectures.to_csv('context/raw_text.csv', header=True, index=False)

In [34]:
def resolve(doc):
    try:
        sents = tokenize.sent_tokenize(doc)
        chunk = 15
        paras = [sents[i*chunk:(i+1)*chunk] for i in range(len(sents)//chunk + 1)]
        for i in range(len(paras)):
            para = ' '.join(paras[i])
            if len(para) != 0:
                paras[i] = predictor.coref_resolved(para)
            else:
                paras[i] = para
        return ' '.join(paras)
    except:
        return 'ERROR'

In [20]:
start = time.time()
resolve(lectures['text'].iloc[0])
print(time.time() - start)

49.96423125267029


In [35]:
chunksize = 20
num = 0
for chunk in pd.read_csv('context/raw_text.csv', chunksize=chunksize, iterator=True):
    chunk['text_resolved'] = chunk['text'].apply(resolve)
    chunk.to_csv(f'tmp/lecture_chunk_{num}.csv', index=False, header=True)
    print(f'Finished chunk {num}')
    num += 1
chunks = []
for i in range(num):
    chunks.append(pd.read_csv(f'tmp/lecture_chunk_{i}.csv'))
chunks = pd.concat(chunks, sort=False, ignore_index=True)
chunks.to_csv('context/lectures_resolved.csv', index=False, header=True)

Finished chunk 0
Finished chunk 1
Finished chunk 2
Finished chunk 3
Finished chunk 4
Finished chunk 5
Finished chunk 6
Finished chunk 7
Finished chunk 8
Finished chunk 9
Finished chunk 10
Finished chunk 11
Finished chunk 12
Finished chunk 13
Finished chunk 14
Finished chunk 15
Finished chunk 16
Finished chunk 17
Finished chunk 18
Finished chunk 19
Finished chunk 20
Finished chunk 21
Finished chunk 22
Finished chunk 23
Finished chunk 24
Finished chunk 25
Finished chunk 26
Finished chunk 27
Finished chunk 28
Finished chunk 29
Finished chunk 30
Finished chunk 31
Finished chunk 32
Finished chunk 33
Finished chunk 34
Finished chunk 35
Finished chunk 36
Finished chunk 37
Finished chunk 38
Finished chunk 39
Finished chunk 40
Finished chunk 41
Finished chunk 42


In [36]:
lectures = pd.read_csv('context/lectures_resolved.csv')

In [ ]:
print(lectures[]

In [28]:
chunks.head()

,text,label,text_resolved
0,The following content is\nprovided under a Cre...,Calculus,The following content is\nprovided under a Cre...
1,"In this sequence of segments,\nwe review some ...",Probability,"In this sequence of segments,\nwe review some ..."
2,The following content is\nprovided under a Cre...,CS,The following content is\nprovided under a Cre...
3,The following\ncontent is provided under a Cre...,Algorithms,The following\ncontent is provided under a Cre...
4,The following\ncontent is provided under a Cre...,Algorithms,The following\ncontent is provided under a Cre...


In [30]:
print(chunks['text'].iloc[0])

The following content is
provided under a Creative Commons license. Your support will help
MIT OpenCourseWare continue to offer high quality
educational resources for free. To make a donation, or to
view additional materials from hundreds of MIT courses,
visit MIT OpenCourseWare at ocw.mit.edu. PROFESSOR: So,
Professor Jerison is relaxing in sunny
London, Ontario today and sent me in as
his substitute again. I'm glad to the here
and see you all again. So our agenda today: he
said that he'd already talked about power series
and Taylor's formula, I guess on last week
right, on Friday? So I'm going to go a
little further with that and show you some examples,
show you some applications, and then I have this
course evaluation survey that I'll hand out in the last
10 minutes or so of the class. I also have this handout
that he made that says 18.01 end of term 2007. If you didn't pick this up
coming in, grab it going out. People tend not to pick it
up when they walk in, I see. So grab this wh

In [37]:
print(lectures['text_resolved'].iloc[0])

The following content is provided under a Creative Commons license. Your support will help MIT OpenCourseWare continue to offer high quality educational resources for free. To make a donation, or to view additional materials from hundreds of MIT OpenCourseWare courses, visit MIT OpenCourseWare at ocw.mit.edu. PROFESSOR: So, Professor Jerison is relaxing in sunny London, Ontario today and sent me in as Professor Jerison's substitute again. me'm glad to the here and see you all again. So our agenda today: Professor Jerison said that Professor Jerison'd already talked about power series and Taylor's formula, me guess on last week right, on Friday? So me'm going to go a little further with that and show you some examples, show you some applications, and then me have this course evaluation survey that me'll hand out in the last 10 minutes or so of the class. me also have this handout that Professor Jerison made that says 18.01 end of term 2007. If you didn't pick this handout that he made t